# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 28 2022 1:53PM,247509,10,0086019013,ISDIN Corporation,Released
1,Sep 28 2022 1:53PM,247509,10,0086021247,ISDIN Corporation,Released
2,Sep 28 2022 1:53PM,247509,10,0086021610,ISDIN Corporation,Released
3,Sep 28 2022 1:53PM,247509,10,0086021613,ISDIN Corporation,Released
4,Sep 28 2022 1:53PM,247509,10,0086021638,ISDIN Corporation,Released
5,Sep 28 2022 1:53PM,247509,10,0086022092,ISDIN Corporation,Released
6,Sep 28 2022 1:53PM,247509,10,0086022090,ISDIN Corporation,Released
7,Sep 28 2022 1:53PM,247509,10,0086023293,ISDIN Corporation,Released
8,Sep 28 2022 1:53PM,247509,10,0086023292,ISDIN Corporation,Released
9,Sep 28 2022 1:53PM,247509,10,0086023305,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,247494,Released,1
21,247506,Released,16
22,247507,Released,31
23,247508,Released,1
24,247509,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247494,NaN,NaN,1.0
247506,NaN,NaN,16.0
247507,NaN,NaN,31.0
247508,NaN,NaN,1.0
247509,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247411,0.0,0.0,1.0
247423,0.0,0.0,1.0
247436,2.0,2.0,1.0
247458,0.0,0.0,1.0
247460,0.0,19.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247411,0,0,1
247423,0,0,1
247436,2,2,1
247458,0,0,1
247460,0,19,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247411,0,0,1
1,247423,0,0,1
2,247436,2,2,1
3,247458,0,0,1
4,247460,0,19,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247411,,,1
1,247423,,,1
2,247436,2,2,1
3,247458,,,1
4,247460,,19,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 28 2022 1:53PM,247509,10,ISDIN Corporation
11,Sep 28 2022 1:51PM,247508,19,"Granules Pharmaceuticals, Inc."
12,Sep 28 2022 1:51PM,247507,10,ISDIN Corporation
43,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd"
59,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc."
60,Sep 28 2022 12:11PM,247493,22,"NBTY Global, Inc."
61,Sep 28 2022 12:09PM,247492,22,"NBTY Global, Inc."
62,Sep 28 2022 11:31AM,247486,10,Emerginnova
64,Sep 28 2022 11:25AM,247485,10,Eywa Pharma Inc.
65,Sep 28 2022 11:06AM,247481,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 28 2022 1:53PM,247509,10,ISDIN Corporation,,,11
1,Sep 28 2022 1:51PM,247508,19,"Granules Pharmaceuticals, Inc.",,,1
2,Sep 28 2022 1:51PM,247507,10,ISDIN Corporation,,,31
3,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",,,16
4,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc.",,,1
5,Sep 28 2022 12:11PM,247493,22,"NBTY Global, Inc.",,,1
6,Sep 28 2022 12:09PM,247492,22,"NBTY Global, Inc.",,,1
7,Sep 28 2022 11:31AM,247486,10,Emerginnova,,1,1
8,Sep 28 2022 11:25AM,247485,10,Eywa Pharma Inc.,,,1
9,Sep 28 2022 11:06AM,247481,20,"ACI Healthcare USA, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 1:53PM,247509,10,ISDIN Corporation,11,,
1,Sep 28 2022 1:51PM,247508,19,"Granules Pharmaceuticals, Inc.",1,,
2,Sep 28 2022 1:51PM,247507,10,ISDIN Corporation,31,,
3,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",16,,
4,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc.",1,,
5,Sep 28 2022 12:11PM,247493,22,"NBTY Global, Inc.",1,,
6,Sep 28 2022 12:09PM,247492,22,"NBTY Global, Inc.",1,,
7,Sep 28 2022 11:31AM,247486,10,Emerginnova,1,1,
8,Sep 28 2022 11:25AM,247485,10,Eywa Pharma Inc.,1,,
9,Sep 28 2022 11:06AM,247481,20,"ACI Healthcare USA, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 1:53PM,247509,10,ISDIN Corporation,11,,
1,Sep 28 2022 1:51PM,247508,19,"Granules Pharmaceuticals, Inc.",1,,
2,Sep 28 2022 1:51PM,247507,10,ISDIN Corporation,31,,
3,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",16,,
4,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 1:53PM,247509,10,ISDIN Corporation,11.0,NaN,NaN
1,Sep 28 2022 1:51PM,247508,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
2,Sep 28 2022 1:51PM,247507,10,ISDIN Corporation,31.0,NaN,NaN
3,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",16.0,NaN,NaN
4,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 28 2022 1:53PM,247509,10,ISDIN Corporation,11.0,0.0,0.0
1,Sep 28 2022 1:51PM,247508,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
2,Sep 28 2022 1:51PM,247507,10,ISDIN Corporation,31.0,0.0,0.0
3,Sep 28 2022 1:43PM,247506,19,"AdvaGen Pharma, Ltd",16.0,0.0,0.0
4,Sep 28 2022 12:11PM,247494,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2474691,73.0,4.0,0.0
15,247460,2.0,19.0,0.0
16,247480,1.0,0.0,0.0
19,989925,18.0,4.0,2.0
20,247481,0.0,1.0,0.0
22,742479,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2474691,73.0,4.0,0.0
1,15,247460,2.0,19.0,0.0
2,16,247480,1.0,0.0,0.0
3,19,989925,18.0,4.0,2.0
4,20,247481,0.0,1.0,0.0
5,22,742479,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,73.0,4.0,0.0
1,15,2.0,19.0,0.0
2,16,1.0,0.0,0.0
3,19,18.0,4.0,2.0
4,20,0.0,1.0,0.0
5,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,15,Released,2.0
2,16,Released,1.0
3,19,Released,18.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,22
Status,,,,,,
Completed,0.0,0.0,0.0,2.0,0.0,0.0
Executing,4.0,19.0,0.0,4.0,1.0,0.0
Released,73.0,2.0,1.0,18.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,22
0,Completed,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,4.0,19.0,0.0,4.0,1.0,0.0
2,Released,73.0,2.0,1.0,18.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,22
0,Completed,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,4.0,19.0,0.0,4.0,1.0,0.0
2,Released,73.0,2.0,1.0,18.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()